In [ ]:
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt


from sklearn.metrics import mean_squared_error, mean_absolute_error

from airpollution_trf_graph_loader import AirpollutionDatasetLoader

pd.options.display.float_format = '{:,.3f}'.format

### Auxiliary functions

In [ ]:
def mape_fn(actual, pred):
    #print(actual)
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / (actual+0.01))) * 100

def compute_metrics_as_dataframe_fn(y_valid, y_hat, particle_name):
    metrics= []
    metrics_global = {'mse':[],'rmse':[],'mae':[],'cvrmse':[],'mape':[],}

    try:
        mae = mean_absolute_error(y_valid, y_hat)
        mse = mean_squared_error(y_valid, y_hat)
        rmse= mean_squared_error(y_valid, y_hat, squared = False)
        cvrmse =  (rmse/np.mean(y_valid))*100 # it is a percentage
        mape = mape_fn(y_valid, y_hat)

        metrics.append((time_horizon, str(particle_name), mae, mse, rmse, cvrmse, mape))


        metrics_df = pd.DataFrame.from_records(metrics, columns='T particle MAE MSE RMSE CVRMSE MAPE'.split())

        return metrics_df
    except:
        return None

def compute_metrics(y_valid, y_hat):
    metrics= []
    metrics_global = {'mse':[],'rmse':[],'mae':[],'cvrmse':[],'mape':[],}

    mae = mean_absolute_error(y_valid, y_hat)
    mse = mean_squared_error(y_valid, y_hat)
    rmse= mean_squared_error(y_valid, y_hat, squared = False)
    cvrmse =  (rmse/np.mean(y_valid))*100 # it is a percentage
    mape = mape_fn(y_valid, y_hat)

    return mae, mse, rmse, cvrmse, mape

In [ ]:
T_lst= [6, 12,24,48] #target time horizons to analyze
city_lst= 'madrid bilbao'.split()
_include_trf= True # include or not traffic data as input

results_path='results'

for _city in city_lst:
    print("\n")
    print("*"*24)

    print(_city)
    loader= AirpollutionDatasetLoader(_city, _include_trf)
    dataset=loader.get_dataset(T=T_lst[0])

    feature_dim= loader.get_feature_dim()
    target_nodes= list(feature_dim.keys())

    y_hat_dict= {}
    y_true_dict= {}
    
    for _trf_str in ['trf', 'no_trf']:    
        for _T in T_lst:
            for k in target_nodes:
                if (_trf_str !=  'no_trf') or (k != 'trf'):
                    _df= pd.read_csv(os.path.join(results_path,f'y_hat_{_city}_{_T}_{k}_{_trf_str}.csv'), index_col=0)
                    y_hat_dict['_'.join([str(_T),k,_trf_str])]= _df
                
                    _df= pd.read_csv(os.path.join(results_path,f'y_true_{_city}_{_T}_{k}_{_trf_str}.csv'), index_col=0)
                    y_true_dict['_'.join([str(_T),k,_trf_str])]= _df


    metrics_by_sensors= []
    metrics_by_pollutants= []
    for _trf_str in ['trf', 'no_trf']:    
        for _T in T_lst:
            for k in target_nodes:
                if k != 'trf':
                    y_true_df= y_true_dict['_'.join([str(_T),k,_trf_str])]
                    y_hat_df= y_hat_dict['_'.join([str(_T),k,_trf_str])]
                
                    #Metris by station
                    for i in range(y_true_df.shape[0]):
                        mae, mse, rmse, cvrmse, mape= compute_metrics(y_true_df.iloc[i], y_hat_df.iloc[i])
                        #print(y_true_df, y_hat_df, mae, mse, rmse, cvrmse, mape)
                        metrics_by_sensors.append((_T, _trf_str, k, i, mae, mse, rmse, cvrmse, mape))
                
                    for c in y_true_df.columns:
                        c_hat= y_hat_df[c].T
                        c_true= y_true_df[c].T
                        mae, mse, rmse, cvrmse, mape= compute_metrics(c_true, c_hat)
                        metrics_by_pollutants.append((_T, _trf_str, k, c, mae, mse, rmse, cvrmse, mape))
    
    
    metrics_by_sensors_df = pd.DataFrame.from_records(metrics_by_sensors, columns='T traffic sensor t MAE MSE RMSE CVRMSE MAPE'.split())
    metrics_by_pollutants_df = pd.DataFrame.from_records(metrics_by_pollutants, columns='T traffic sensor pollutant MAE MSE RMSE CVRMSE MAPE'.split())
    
    metrics_by_sensors_df.to_csv(os.path.join(results_path,f'metrics_by_sensor_{_city}.csv'))
    metrics_by_pollutants_df.to_csv(os.path.join(results_path,f'metrics_by_pollutant_{_city}.csv'))

    print("*"*8)
    print("METRICS BY POLLUTANT")
    for t in T_lst:
        print("*"*4)
        print(f"{_city} - {t}h time horizon")
        metric_agg_mean_df= metrics_by_pollutants_df[metrics_by_pollutants_df['T']==t].drop(columns='sensor').groupby('pollutant T traffic'.split()).mean()
        print(metric_agg_mean_df)

    print("*"*8)
    print("METRICS BY SENSORS")
    for t in T_lst:
        print(f"{_city} - {t}h time horizon")
        metric_agg_mean_df= metrics_by_sensors_df[metrics_by_sensors_df['T']==t].groupby('sensor T traffic'.split()).mean()
        print(metric_agg_mean_df)  
        print("*"*4)  

    print("*"*8)
    print("FORECASTED VS REAL PLOTS")

    T_to_plot= '24'
    if _city== 'bilbao':
        T_to_plot= '12'
    y_true_lst={}
    y_hat_lst={}
    for k,df in y_true_dict.items():
        T_k= k.split('_')[0]
        trf_k= k.split('_')[2]
        s_k = k.split('_')[1]
        if (T_to_plot == T_k) and (trf_k== 'trf') and (s_k != 'trf'): # We plot pollution sensors including traffic as input
            y_hat_df= y_hat_dict[k]
            for c in df.columns:
                df_lst= y_true_lst.get(c,[])
                df_lst.append(df[c].to_frame().reset_index())
                y_true_lst[c]= df_lst
    
                df_lst= y_hat_lst.get(c,[])
                df_lst.append(y_hat_df[c].to_frame().reset_index())
                y_hat_lst[c]= df_lst
    
    for k, lst in y_true_lst.items():
        
        df= pd.concat(lst, axis=0)
        df= df.groupby('index').mean()
        df= df.rename(columns={k:f'{k}_true'})
    
        lst2= y_hat_lst[k]
        y_hat_df= pd.concat(lst2, axis=0)
        y_hat_df= y_hat_df.groupby('index').mean()
        y_hat_df= y_hat_df.rename(columns={k:f'{k}_forecasted'})
        
        ax=df.plot(grid=True, figsize=(15,5));
        y_hat_df.plot(ax=ax, grid=True, xlabel='Snapshot', title=f'{_city} {k}');
        
        plt.savefig(os.path.join(os.path.join('figs', f'true_vs_forecast_{k}_{T_to_plot}_{_city}.png')), bbox_inches='tight')

In [ ]:
print("That's all folks!")

# Test code (do not run)

In [ ]:
from scipy.stats import ttest_ind

T_lst= [12,24] #target time horizons to analyze
_city= 'madrid'
_include_trf= True # include or not traffic data as input

results_path='results'

loader= AirpollutionDatasetLoader(_city, _include_trf)
dataset=loader.get_dataset(T=T_lst[0])

feature_dim= loader.get_feature_dim()
feature_dim

In [ ]:
target_nodes= list(feature_dim.keys())
target_nodes

In [ ]:
y_hat_dict= {}
y_true_dict= {}

for _trf_str in ['trf', 'no_trf']:    
    for _T in T_lst:
        for k in target_nodes:
            if (_trf_str !=  'no_trf') or (k != 'trf'):
                _df= pd.read_csv(os.path.join(results_path,f'y_hat_{_city}_{_T}_{k}_{_trf_str}.csv'), index_col=0)
                y_hat_dict['_'.join([str(_T),k,_trf_str])]= _df
            
                _df= pd.read_csv(os.path.join(results_path,f'y_true_{_city}_{_T}_{k}_{_trf_str}.csv'), index_col=0)
                y_true_dict['_'.join([str(_T),k,_trf_str])]= _df

In [ ]:
y_true_dict

In [ ]:
metrics_by_sensors= []
metrics_by_pollutants= []
for _trf_str in ['trf', 'no_trf']:    
    for _T in T_lst:
        for k in target_nodes:
            if k != 'trf':
                y_true_df= y_true_dict['_'.join([str(_T),k,_trf_str])]
                y_hat_df= y_hat_dict['_'.join([str(_T),k,_trf_str])]
            
                #Metris by station
                for i in range(y_true_df.shape[0]):
                    mae, mse, rmse, cvrmse, mape= compute_metrics(y_true_df.iloc[i], y_hat_df.iloc[i])
                    #print(y_true_df, y_hat_df, mae, mse, rmse, cvrmse, mape)
                    metrics_by_sensors.append((_T, _trf_str, k, i, mae, mse, rmse, cvrmse, mape))
            
                for c in y_true_df.columns:
                    c_hat= y_hat_df[c].T
                    c_true= y_true_df[c].T
                    mae, mse, rmse, cvrmse, mape= compute_metrics(c_true, c_hat)
                    metrics_by_pollutants.append((_T, _trf_str, k, c, mae, mse, rmse, cvrmse, mape))


metrics_by_sensors_df = pd.DataFrame.from_records(metrics_by_sensors, columns='T traffic sensor t MAE MSE RMSE CVRMSE MAPE'.split())
metrics_by_pollutants_df = pd.DataFrame.from_records(metrics_by_pollutants, columns='T traffic sensor pollutant MAE MSE RMSE CVRMSE MAPE'.split())

metrics_by_sensors_df.to_csv(os.path.join(results_path,f'metrics_by_sensor_{_city}.csv'))
metrics_by_pollutants_df.to_csv(os.path.join(results_path,f'metrics_by_pollutant_{_city}.csv'))

### Metrics by sensor

In [ ]:
metrics_by_sensors_df

12 hours horizon

In [ ]:
metric_agg_mean_df= metrics_by_sensors_df[metrics_by_sensors_df['T']==12].groupby('sensor T traffic'.split()).mean()
metric_agg_mean_df

In [ ]:
metric_agg_mean_df= metrics_by_sensors_df[metrics_by_sensors_df['T']==12].groupby('sensor T traffic t'.split()).mean().reset_index()
metric_agg_mean_df[metric_agg_mean_df['t']==0]

In [ ]:
metric_agg_mean_df[metric_agg_mean_df['t']==6]

24 hours horizon

In [ ]:
metric_agg_mean_df= metrics_by_sensors_df[metrics_by_sensors_df['T']==24].groupby('sensor T traffic'.split()).mean()
metric_agg_mean_df

In [ ]:
metric_agg_mean_df.reset_index()

In [ ]:
metric_agg_mean_df= metric_agg_mean_df.reset_index()

In [ ]:
metric_trf= metric_agg_mean_df[metric_agg_mean_df['traffic']=='trf']['RMSE']
metric_no_trf= metric_agg_mean_df[metric_agg_mean_df['traffic']=='no_trf']['RMSE']
metric_trf, metric_no_trf

In [ ]:
ttest_ind(metric_trf, metric_no_trf)

## Metrics by pollutants

12 hours horizon

In [ ]:
metric_agg_mean_df= metrics_by_pollutants_df[metrics_by_pollutants_df['T']==12].drop(columns='sensor').groupby('pollutant T traffic'.split()).mean()
metric_agg_mean_df

24 hours horizon

In [ ]:
metric_agg_mean_df= metrics_by_pollutants_df[metrics_by_pollutants_df['T']==24].drop(columns='sensor').groupby('pollutant T traffic'.split()).mean()
metric_agg_mean_df

In [ ]:
metrics_by_pollutants_df

In [ ]:
for c in metrics_by_pollutants_df['pollutant'].unique():
    for m in 'MAE RMSE CVRMSE MAPE'.split():
        metric_trf= metrics_by_pollutants_df[(metrics_by_pollutants_df['traffic']=='trf') & 
        (metrics_by_pollutants_df['pollutant']==c)][m]
        
        metric_no_trf= metrics_by_pollutants_df[(metrics_by_pollutants_df['traffic']=='no_trf') & 
        (metrics_by_pollutants_df['pollutant']==c)][m]
        print(c,m)
        print(ttest_ind(metric_trf, metric_no_trf, equal_var=False))

## Plot forecasted vs real values

In [ ]:
T_to_plot='24'
y_true_lst={}
y_hat_lst={}
for k,df in y_true_dict.items():
    y_hat_df= y_hat_dict[k]
    if (T_to_plot in k) and ('no_trf' in k):
        for c in df.columns:
            df_lst= y_true_lst.get(c,[])
            df_lst.append(df[c].to_frame().reset_index())
            y_true_lst[c]= df_lst

            df_lst= y_hat_lst.get(c,[])
            df_lst.append(y_hat_df[c].to_frame().reset_index())
            y_hat_lst[c]= df_lst

In [ ]:
_trf_str= 'trf'
if not _include_trf:
    _trf_str='no_trf'

for k, lst in y_true_lst.items():
    
    df= pd.concat(lst, axis=0)
    df= df.groupby('index').mean()
    df= df.rename(columns={k:f'{k}_true'})

    lst2= y_hat_lst[k]
    y_hat_df= pd.concat(lst2, axis=0)
    y_hat_df= y_hat_df.groupby('index').mean()
    y_hat_df= y_hat_df.rename(columns={k:f'{k}_forecasted'})
    
    ax=df.plot(grid=True, figsize=(15,5));
    y_hat_df.plot(ax=ax, grid=True, xlabel='Snapshot');
    

    plt.savefig(os.path.join(os.path.join('figs', f'true_vs_forecast_{k}_{T_to_plot}_{_city}_{_trf_str}.png')), bbox_inches='tight')